# Median with Spark
## using .cartesian()

In [2]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [35]:
c = sc.parallelize([2,3,3,4,4,5,5,5,6,6])

def count_greater_minus_lower(x):
    a, b = x
    if a[0] < b[0]:
        return + b[1]
    if a[0] > b[0]:
        return - b[1]
    return 0

#count number of occurrences for each value (value, count)

get_freq = c.map(lambda x: (x, +1))\
    .reduceByKey(lambda l, r: l+r)

get_median = get_freq.cartesian(get_freq)\
    .map(lambda x: (x[0], count_greater_minus_lower(x)))\
    .reduceByKey(lambda l, r: l+r)\
    .map(lambda x: (x[0][0], (x[0][1], x[1])))\
    .filter(lambda x: abs(x[1][1]) <= x[1][0])\
    .keys().collect()

"""
state of the tuples after each operation:
after cartesian: ((value_1, count_1), (value_2, count_2))
after map: ((value_1, count_1), +/- count_2) : +count_2 if value_2 > value_1, -count_2 else
after reduce: ((value, count), diff) where diff is the difference between counts of numbers greater than value and 
                                    numbers lower than value
after map: (value, (count, diff))
to compute the median I need those values such that diff <= count
"""

print(sum(get_median)/len(get_median))

4.5
